In [33]:
import pandas as pd
import json
import cv2
import os
import math
import datetime
import numpy as np
from PIL import Image
from IPython.display import clear_output
import json 
import random

In [34]:
# Variables.
# ------------------------------------------

who_is_running_this_code = "hogni"

file_name = "edi_30fps_nosudio_full"
parent_parth = f"/Users/{who_is_running_this_code}/Library/Mobile Documents/com~apple~CloudDocs/Bachelor Project/Videos/09032021/"

counter_path = f"{parent_parth}Data/{file_name}/counterData_{file_name}.csv"
tracker_path = f"{parent_parth}Data/{file_name}/tracker{file_name}.csv"
video_path = f"{parent_parth}Processed/{file_name}.mp4"
photo_path = f"{parent_parth}Photos/{file_name}"
save_labels_path = f"{parent_parth}Data/{file_name}/"
db_path = f"{parent_parth}Data/{file_name}/{file_name}_db.csv"
distance = 2.6
orientation = 130

In [35]:
# Velocity Calculations
# ------------------------------------------

def calculate_velocity(distance, df):
    """Velocity Calculations: v = S/t """
    velocit_dictionary = {}
    for index, row in df.iterrows():
        if row[4] in velocit_dictionary.keys():
            velocit_dictionary[row[4]].append(row[1])
        else:
            velocit_dictionary[row[4]] = [row[1]]

    for key, value in velocit_dictionary.copy().items():
        if value[0] > value[1]:
            difference = (value[0]-value[1])
        else:
            difference = (value[1]-value[0])
        velocit_dictionary[key] = difference

    for key, value in velocit_dictionary.copy().items():
        t = velocit_dictionary[key].total_seconds()
        if t == 0:
            t = 1
            velocit_dictionary[key] = distance/t
        else:
            velocit_dictionary[key] = distance/t
    return velocit_dictionary

# Absolute Heading 
# ------------------------------------------

def calculate_true_heading(frame_orientation, df):
    true_heading_dictionary = {}
    for index, row in df.iterrows():
        true_heading = row[5] + frame_orientation
        if true_heading > 360:
            true_heading = true_heading - 360
        true_heading_dictionary[row[0]] = true_heading
    return true_heading_dictionary

# Init DataFrames
# ------------------------------------------

# Counter
def create_counter_df(path, distance, orientation):
    """Read and format counter df"""
    columns = ["frameId", "timestamp", "counter_area", "ObjectClass", 
            "UniqueID", "bearing_og", "countingDirection", "angle"]

    counter = pd.read_csv(path, names=columns)
    counter = counter[counter.duplicated(subset=["UniqueID"], keep=False)].sort_values("UniqueID")
    counter['timestamp'] = pd.to_datetime(counter['timestamp'], format='%Y-%m-%dT%H:%M:%S.%fZ')
    counter = fix_df(counter)
    counter = add_velocity_to_df(distance, counter)
    counter = add_true_heading_to_df(orientation, counter)
    return counter

def add_velocity_to_df(distance, counter_df):
    """ Calculates speeds (and append to df) speeds: {UniqueID: velocity} """
    speeds = calculate_velocity(distance, counter_df)
    for key, value in speeds.items():
        counter_df.loc[counter_df["UniqueID"]==key, "velocity"]=value
    return counter_df

def add_true_heading_to_df(orientation, counter_df):
    """ Calculates true heading (and append to df): {frameId: true_heading} """
    true_heading = calculate_true_heading(orientation, counter_df)
    for key, value in true_heading.items():
        counter_df.loc[counter_df["frameId"]==key, "true_heading"]=value
    return counter_df

# Tracker
def create_tracker_df(tracker_path):
    with open(tracker_path) as f:
        tracker = json.load(f)
    tracker_flattend = pd.json_normalize(tracker, record_path='objects', meta=['frameId']).rename(columns={'id': 'UniqueID'})
    tracker_flattend = tracker_flattend.drop_duplicates(subset=['UniqueID', 'frameId'], keep='first')
    return tracker_flattend

# Fix inaccurate data
# ------------------------------------------

def fix_df(counter_df):
    error_dict = {}
    for index, row in counter_df.iterrows():
        if row["UniqueID"] in error_dict:
            error_dict[row["UniqueID"]].append([row["frameId"], row["bearing_og"]])
        else:
            error_dict[row["UniqueID"]] = [[row["frameId"], row["bearing_og"]]]

    for key, value in error_dict.items():
        if value[0][1] == 270 or value[1][1] == 270:
            if value[0][1] == 270:
                if value[1][1] < 150 and value[1][1] > 50:
                   counter_df.loc[counter_df["frameId"]==error_dict[key][0][0], "bearing_og"]=90
            else:
                if value[0][1] < 150 and value[0][1] > 50:
                   counter_df.loc[counter_df["frameId"]==error_dict[key][1][0], "bearing_og"]=90
    return counter_df

# Merge counter and tracker
# ------------------------------------------

def merge_counter_and_tracker_df(counter_path, tracker_path, distance, orientation):
    counter_df = create_counter_df(counter_path, distance, orientation)
    tracker_df = create_tracker_df(tracker_path)
    return pd.merge(counter_df, tracker_df, on=["UniqueID", "frameId"], how="left")

In [36]:
# Image Generation
# ------------------------------------------

def save_frame(frame_number, source, arrows=None):
    vc = cv2.VideoCapture(source)
    vc.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    rval, frame = vc.read()
    if arrows != None:
        for a in arrows:
            frame = cv2.arrowedLine(frame, a['start'], a['end'], (0,0,255), thickness=8, tipLength=0.6)
        cv2.imwrite(f"{photo_path}/{str(frame_number)}.jpg", frame)

def get_arrow(obj):
    x, y = obj['x'], obj['y']
    angle = obj['bearing_og']
    speed = obj['velocity']
    a = 30 * speed * math.sin(math.radians(angle))
    b = 30 * speed * math.cos(math.radians(angle))
    return {'start': (int(x), int(y)), 'end': (int(x+a), int(y+b))}

# Stich images together.
# ------------------------------------------

def get_image_pairs(merged_df):
    image_pairs_dict = {}
    for index, row in merged_df.iterrows():
        if row[4] in image_pairs_dict:
            image_pairs_dict[row["UniqueID"]].append(row["frameId"])
        else:
            image_pairs_dict[row["UniqueID"]] = [row["frameId"]]
    return image_pairs_dict

def merge_images(photo_path):
    image_pairs_dict = get_image_pairs(merged_df)
    for key, value in image_pairs_dict.items():
        image1 = Image.open(f"{photo_path}/{value[0]}.jpg")
        image2 = Image.open(f"{photo_path}/{value[1]}.jpg")

        (width1, height1) = image1.size

        result_width = width1 * 2
        result_height = height1

        result = Image.new('RGB', (result_width, result_height))
        result.paste(im=image1, box=(0, 0))
        result.paste(im=image2, box=(width1, 0))
        result.save(f"{photo_path}/Merged/{key}.jpg")

def make_all_images(video_path, photo_path, merged_df):
    for _, row in merged_df.iterrows():
        save_frame(row['frameId'], video_path, arrows=[get_arrow(row)])
    merge_images(photo_path)
    return "Completed"

### Make df and pictures.

In [37]:
# Make df and pictures
#merged_df = merge_counter_and_tracker_df(counter_path, tracker_path, distance, orientation)

In [38]:
#make_all_images(video_path, photo_path, merged_df)

## Tag pictures

In [39]:
def show_pictures(photo_path, unique_id):
    path = f"{photo_path}/Merged"
    img = Image.open(f"{path}/{unique_id}.jpg")
    return display(img)

def label_photos(photo_path, merged_df):
    unique_id_set = set()
    labels_dict = {}
    for index, row in merged_df.iterrows():
        unique_id_set.add(row["UniqueID"])
    
    for i in unique_id_set:
        show_pictures(photo_path, i)
        path = int(input("Path: "))
        mode = int(input("Mode: "))
        labels_dict[i] = (path, mode)
        clear_output(wait=True)
    return labels_dict

In [40]:
# Add tags to df
# ------------------------------------------

def add_labels_df(labels, merged_df):
    for key, value in labels.items():
        merged_df.loc[merged_df["UniqueID"]==key, "path"]=value[0]
        merged_df.loc[merged_df["UniqueID"]==key, "mode"]=value[1]
    return merged_df

def save_labels(labels, file_name, save_labels_path):
    json_file = json.dumps(labels)
    f = open(f"labels_{file_name}.json","w")
    f.write(json_file)
    f.close()
    return "saved"

In [41]:
#labels = label_photos(photo_path, merged_df)

In [42]:
#merged_df = add_labels_df(labels, merged_df)

### Save df and labels

In [43]:
#save_labels(labels, file_name, save_labels_path)

In [44]:
#merged_df.to_csv(db_path)

###  Load saved DF

In [45]:
merged_df = pd.read_csv(db_path)

In [46]:
#f = open(f"{save_labels_path}labels_{file_name}.json",)
#labels = json.load(f) 
#f.close() 

In [47]:
#merged_df = add_labels_df(labels, merged_df)

# Generate fake data

In [48]:
def gen_data(X_vector_list, y_vector_list, n_data_point):
    """ Return db of generated data """
    #"bearing_1", "bearing_1", "velocity"
    temp_x = []
    temp_y = []
    outer_loop_range = round(n_data_point/len(X_vector_list))
    for i in range(outer_loop_range):
        for index, a in enumerate(X_vector_list):
            noise = random.randrange(1, 5, 1)
            noise = noise + random.random()
            temp_x.append([a[0] + noise, a[1] + noise, a[2]])
            temp_y.append(y_vector_list[index])
    return temp_x, temp_y

## Tensorflow FNN

In [49]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [50]:
def create_xy_data(df):
    X_vector_list = []
    y_vector_list = []
    temp_vector = []
    temp_id = 0
    for index, row in df.iterrows():
        if temp_id == 0:
            temp_id = row["UniqueID"]
            temp_vector.append(row["bearing"])
            #zeros = [0,0,0,0,0,0,0]
            zeros = [0,0,0,0,0]
            zeros[int(row["path"])-1] = 1
            #zeros[int(-row["mode"])] = 1
            y_vector_list.append(zeros)
        else:
            if row["UniqueID"] == temp_id:
                temp_vector.append(row["bearing"])
                temp_vector.append(row["velocity"])
        if len(temp_vector) == 3:
            X_vector_list.append(temp_vector)
            temp_vector = []
            temp_id = 0
    return X_vector_list, y_vector_list

In [51]:
# Real Data
#X_train, X_test, y_train, y_test = train_test_split(X_vector_list, y_vector_list, test_size=0.33, random_state=42)
X_vector_list, y_vector_list= create_xy_data(merged_df)

# Fake Data
X_train, y_train = gen_data(X_vector_list, y_vector_list, 1000)

In [52]:
print(len(X_train))
print(len(X_vector_list))

1064
152


In [53]:
X_train = tf.constant(X_train, dtype=tf.float32)
X_test = tf.constant(X_vector_list, dtype=tf.float32)

y_train = tf.constant(y_train, dtype=tf.int16)
y_test = tf.constant(y_vector_list, dtype=tf.int16)

In [54]:
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(3,)),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [55]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 20)                80        
_________________________________________________________________
dense_4 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 55        
Total params: 345
Trainable params: 345
Non-trainable params: 0
_________________________________________________________________


In [56]:
loss_fn = tf.keras.losses.BinaryCrossentropy()

In [57]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [58]:
model.fit(X_train, y_train, epochs=15)

Epoch 1/15
34/34 [==============================] - 0s 481us/step - loss: 11.8828 - accuracy: 0.5538
Epoch 2/15
34/34 [==============================] - 0s 468us/step - loss: 2.6995 - accuracy: 0.3421
Epoch 3/15
34/34 [==============================] - 0s 421us/step - loss: 1.3580 - accuracy: 0.3542
Epoch 4/15
34/34 [==============================] - 0s 413us/step - loss: 0.8543 - accuracy: 0.4657
Epoch 5/15
34/34 [==============================] - 0s 400us/step - loss: 0.6100 - accuracy: 0.5762
Epoch 6/15
34/34 [==============================] - 0s 397us/step - loss: 0.4357 - accuracy: 0.5782
Epoch 7/15
34/34 [==============================] - 0s 389us/step - loss: 0.4138 - accuracy: 0.5978
Epoch 8/15
34/34 [==============================] - 0s 406us/step - loss: 0.3735 - accuracy: 0.6245
Epoch 9/15
34/34 [==============================] - 0s 396us/step - loss: 0.3315 - accuracy: 0.6679
Epoch 10/15
34/34 [==============================] - 0s 412us/step - loss: 0.3367 - accuracy: 0.812

In [59]:
model.evaluate(X_test,  y_test, verbose=2)

5/5 - 0s - loss: 0.2459 - accuracy: 0.9211


[0.2459195852279663, 0.9210526347160339]

In [28]:
#model.predict(X_test)